# Matrix Bubbling

In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

## Some Morphological Operations

Morphological operations apply a structuring element to an input image, creating an output image of the same size. In a morphological operation, the value of each pixel in the output image is based on a comparison of the corresponding pixel in the input image with its neighbors. By choosing the size and shape of the neighborhood, you can construct a morphological operation that is sensitive to specific shapes in the input image.

We'll use the following morphological operations to try to mimick real life transformations to images.

### Dilation and Erosion

Dilation adds pixels to the boundaries of objects in an image, while erosion removes pixels on object boundaries. The number of pixels added or removed from the objects in an image depends on the size and shape of the structuring element used to process the image. In the morphological dilation and erosion operations, the state of any given pixel in the output image is determined by applying a rule to the corresponding pixel and its neighbors in the input image. The rule used to process the pixels defines the operation as a dilation or an erosion. 
Reference - https://www.mathworks.com/help/images/morphological-dilation-and-erosion.html

* 
Dilation - The value of the output pixel is the maximum value of all the pixels in the input pixel's neighborhood. In a binary image, if any of the pixels is set to the value 1, the output pixel is set to 1.

* 
Erosion - The value of the output pixel is the minimum value of all the pixels in the input pixel's neighborhood. In a binary image, if any of the pixels is set to 0, the output pixel is set to 0.

In [6]:
# This file contains all the main external libs we'll use
# We'll assume you already have numpy and opencv installed
# from fastai.imports import *
import cv2
import numpy as np
import matplotlib as plt

# Utility Functions
def draw_with_size(image, size_in_inches):
    plt.figure(figsize=(size_in_inches, size_in_inches))
    plt.imshow(image);
def random_image(shape):
    return np.random.uniform(0, 1, size=shape).astype(np.float32)
def scale_match(src, targ):
    """Returns the targ image resized so that its at exactly as big as src, while maintaining its aspect ratio.
    This may mean that the right or the bottom may be cropped."""
    h,w,_ = src.shape
    sh,sw,_ = targ.shape
    ratio = max(h/sh,w/sw);
    res = cv2.resize(targ, (int(sw*ratio), int(sh*ratio)))
    return res[:h,:w]

#image = open_image('flower.jpg')
img = cv2.imread('flower.jpg',0)
draw_with_size(img, 7)
kernel = np.ones((5,5),np.uint8)
erosion = cv2.erode(img,kernel,iterations = 1)
draw_with_size(img, 7)

TypeError: 'module' object is not callable

## Erosion and Dilation

* 
Erosion, also known by the name “shrink”. It turns off pixels which were near pixels that were off originally, thereby eating away at the edges of the items in the image.

* 
Dilate is a function that accepts a black and white image. It is also known by the names “grow”, “bolden”, and “expand”. It turns on pixels which were near pixels that were on originally, thereby thickening the items in the image.

See https://blog.ostermiller.org/dilate-and-erode for reference

In [2]:
from scipy import ndimage

def erode(image):
    

SyntaxError: unexpected EOF while parsing (<ipython-input-2-9f22a7a8343e>, line 4)